# <center><span style="font-size: 42px;color: darkgreen;">Executando o Apache Spark</span></center>

<br>

---

<br>

# Execução do Apache Spark em Modo Standalone

<br>

### Acessando o Spark via Shell

Existem duas formas de acessar o Spark via shell, dependendo da linguagem que deseja utilizar:

<br>

1. **Acessando o Shell em Scala**

   - No terminal, digite o seguinte comando para abrir o shell com a linguagem **Scala**:

     ```bash
     
     spark-shell
     
     
     ```
    - Para sair do shell Scala, digite:
   
     ```scala

     :q

     ```

<br>

2. **Acessando o Shell em Python**

   - Para abrir o shell do Spark com a linguagem **Python**, primeiro, **ative o ambiente Conda onde o Spark está configurado** (no seu caso, `spark_env`):

       ```bash
       conda activate spark_env
       ```

     - Depois, digite o seguinte comando para acessar o shell do Spark em Python:

       ```bash
       pyspark
       # Uma vez dentro do shell Python do Spark, você pode executar comandos Spark em Python.
       ```  
       
<br>

   - Para sair do shell Python, digite:

     ```python
     exit()
     ```

<br>

> **Observação**: Lembre-se de desativar o ambiente `spark_env` quando não precisar mais do Spark em Python para retornar ao ambiente padrão do sistema. Para isso, use:

   ```bash
   conda deactivate
   ```

<br><br><br>

## Executando Uma Aplicação de Exemplo

<br>

Após desenvolver a aplicação para resolver o problema de negócio (análise ou modelo preditivo), não utilizamos o `spark-shell` ou `pyspark`. Em vez disso, usamos o **`spark-submit`**, um binário específico do Spark que envia o `Job` para execução no cluster Spark.

Para este exemplo **não é necessário ativar ambiente conda** pois usaremos Scala e não o Python.

<br>

#### Verificando Aplicações de Exemplo do Spark

<br>

- **1.** Ir ao diretório home do Spark digitando no terminal:

   ```code
   cd $SPARK_HOME
   ```
   
<br>

- **2.** Dentro do diretório do Spark, ir até o diretório examples e seguir:

   ```code
   cd examples
   cd src
   cd main
   cd python
   ls -la 
   # aqui teremos algumas aplicações prontas para exemplo
   ```   
   
<br>

- **3.** Retornar ao diretório examples e seguir :

   ```code
   cd jars/
   ls -la    
   # dentro do arquivo spark-examples_2.11-2.4.3.jar temos uma aplicação exemplo de como calcular o valor de PI
   ```
   
<br>
   
- **4.** Retonar a pasta home:

   ```code
   cd ~
   ```
   
<br>

- **5.** Executando uma aplicação do Apache Spark (iremos executar uma classe que está dentro do arquivo `spark-examples_2.11-2.4.3.jar`):

   ```code
   spark-submit --class org.apache.spark.examples.SparkPi --master local $SPARK_HOME/examples/jars/spark-examples_2.11-2.4.3.jar
   ```

<br>

- **5.1** Executando **outra** aplicação do Apache Spark (iremos executar uma classe que está dentro do arquivo `spark-examples_2.11-2.4.3.jar`):

   ```code
   spark-submit --class org.apache.spark.examples.LocalKMeans --master local $SPARK_HOME/examples/jars/spark-examples_2.11-2.4.3.jar
   ```
<br>

---

<br>

# <center>Construindo e Executando Aplicação de MapReduce</center>

Neste **exemplo** iremos definir o problema de negócio, gerar a massa de dados, construir a aplicação para o `Apache Spark` e realizar a execução.

<br>

### Problema de Negócio

Desenvolver uma aplicação para o `Apache Spark` que acessa um arquivo de texto, analisa o conteúdo e contabiliza a frequência de ocorrência de cada palavra no documento.

<br><br>

### 1. Ativar Ambiente Conda

Primeiro, **ative o ambiente Conda onde o Spark está configurado** (no seu caso, `spark_env`):

```bash
conda activate spark_env    |    conda deactivate
```

<br>

### 2. Gerar Massa de Dados

Ir ao terminal e digitar:

```bash
gedit input.txt
```

Cole um texto qualquer e salve o arquivo.

<br>

### 3. Desenvolver Aplicação

Escrever o código utilizando a linguagem `Python` e criar o arquivo `app.py`:

<br>

```python
import sys

from pyspark import SparkContext, SparkConf

if __name__ == "__main__":

	# Criar SparkContext
	conf = SparkConf().setAppName("Conta Palavras").set("master", "local")
	sc = SparkContext(conf = conf)


	# Carregar o Arquivo de Texto (está no diretório local, caso estivesse no HDFS deve ser informado)
	palavras = sc.textFile("/home/hadoop/input.txt")

	# Quebrar cada linha do texto em palavras individuais, utilizando o espaço como delimitador
	palavras = palavras.flatMap(lambda line: line.split(" "))


	# Mapear cada palavra para uma contagem de 1
	contagem = palavras.map(lambda palavra: (palavra, 1))

	# Reduzir por chave (palavra) para somar as contagens
	resultado = contagem.reduceByKey(lambda a, b: a + b)

	# Salvar o resultado em um novo arquivo 
	resultado.saveAsTextFile("/home/hadoop/saida")
    
```

<br>

### 3. Executando a Aplicação

Navegar até o diretório do arquivo `app.py` e digitar no terminal:

<br>

```bash
spark-submit app.py
```

<br>

### 4. Verificando Resultado

Navegar até o diretório `saida` criado na etapa 2 e visualize:

```bash
cat /home/hadoop/saida/part-00000
```

<br><br>

---

<br>

<br>

# Execução do Apache Spark Remotamente

<br><br>

## Como Executar Jobs Remotamente no Cluster Spark

<br>

### Contexto

No ambiente corporativo, o acesso aos **Clusters Spark** é frequentemente realizado de maneira **remota**, seja em máquinas de clientes ou em projetos internos. Agora iremos simular esse acesso de forma **remota**, operando no modelo **cliente-servidor**.

Para este exemplo, temos uma **Máquina Virtual** configurada via `Oracle VirtualBox`, pronta para executar produtos do ecossistema `Hadoop`. Na **etapa anterior**, criamos e executamos a aplicação `app.py` diretamente no servidor. Agora, **acessaremos o servidor remotamente a partir da máquina local (física) para executar a mesma aplicação**.

<br>

### Objetivo

Conectar-se remotamente ao servidor e executar a aplicação `app.py` desenvolvida na etapa anterior.

<br><br>

### 1. Configurando Apache Spark na Máquina Virtual

<br>

#### 1.1 Copiando os Arquivos Template

- Ir ao diretório `/opt/spark/conf` e copiar os arquivos `spark-env.sh.template` e `slaves.template` para `spark-env.sh` e `slaves`, respectivamente:

```code
cp spark-env.sh.template spark-env.sh
cp slaves.template slaves
```

<br><br>

#### 1.2 Obtendo o IP da Máquina Local

- Execute o comando `ifconfig` ou `ip addr` na sua **máquina virtual**. Isso exibirá as interfaces de rede ativas e os respectivos endereços IP. O IP usado, no seu caso `192.168...`, é o endereço da rede local que permite que a máquina virtual acesse a máquina física.


```code
ifconfig
```

<br><br>

#### 1.3 Configurando os Arquivos `spark-env.sh` e `slaves`

- No arquivo `spark-env.sh`, adicione as seguintes linhas:

```code
export SPARK_LOCAL_IP=192.168...
export SPARK_MASTER_HOST=192.168...
```

- No arquivo `slaves`, adicione as seguintes linhas:

```code
192.168...
```

<br><br>

#### 1.4 Reinicializando e Testando o Spark

- No terminal, execute o comando no diretório `/opt/spark/sbin` do Spark para iniciar o **Master** e os **Workers**:

```code
./start-all.sh
jps
```

- Direto do diretório home:

```code
/opt/spark/sbin/start-all.sh
jps
```

<br>

- Agora **teste na máquina local** abrindo o navegador e digite:

```code
192.168...:8080
```

<br><br>

### 2. Configurando Apache Spark na Máquina Local

<br>

- **Importante**: É necessário realizar o **download** e fazer a **instalação** do `Apache Spark` da **mesma versão** da **máquina virtual**.

<br>

#### 2.1 Instalando o Spark na Máquina Local

- Navegar até o diretório onde está o arquivo do `Spark` e digitar no terminal:

```code
tar -xzvf spark-2.4.4-bin-hadoop2.7.tgz
sudo mv spark-2.4.4-bin-hadoop2.7 /opt/
sudo mv /opt/spark-2.4.4-bin-hadoop2.7 /opt/spark
```

<br>

- Retornar ao diretório `home` e editar as variáveis de ambiente:

```code
gedit .bashrc

# Colar as linhas:
# Spark
export SPARK_HOME=/opt/spark
export PATH=$PATH:$SPARK_HOME/bin
#export PYSPARK_DRIVER_PYTHON=jupyter
#export PYSPARK_DRIVER_PYTHON_OPTS=notebook
#export PYSPARK_PYTHON=python3

source .bashrc
```
<br>

- Testar conectando na máquina local:

```code
spark-shell
:q
```

<br><br>

### 3. Acesso ao Cluster Spark (Hospedado na Máquina Virtual) a Partir da Máquina Local

<br>

#### 3.1 Estabelecendo Conexão Remota com o Cluster Spark na Máquina Virtual

- No terminal da **máquina virtual** inicie os serviços:

```code
/opt/spark/sbin/start-all.sh
jps
```

<br>

- No terminal da **máquina local** digitar para se conectar:

```code
spark-shell --master spark://192.168.0.19:7077
```

<br>

- Acessar via navegador:

```code
192.168...:8080
```

<br>

### 4. Executando Jobs Remotamente

<br>

- Escrever o código utilizando a linguagem `Python` e criar o arquivo `app_local.py`:

<br>

```python
import sys
from pyspark import SparkContext, SparkConf

if __name__ == "__main__":

    # Criar SparkContext
    conf = SparkConf().setAppName("Conta Palavras").setMaster("spark://192.168.0.19:7077")
    sc = SparkContext(conf=conf)


    # Carregar o Arquivo de Texto (está no diretório local, caso estivesse no HDFS deve ser informado)
    palavras = sc.textFile("/home/eduardo/Compartilhada/input.txt")

    # Quebrar cada linha do texto em palavras individuais, utilizando o espaço como delimitador
    palavras = palavras.flatMap(lambda line: line.split(" "))

    # Mapear cada palavra para uma contagem de 1
    contagem = palavras.map(lambda palavra: (palavra, 1))

    # Reduzir por chave (palavra) para somar as contagens
    resultado = contagem.reduceByKey(lambda a, b: a + b)

    # Salvar o resultado em um novo arquivo 
    resultado.saveAsTextFile("/home/eduardo/Compartilhada/saida10")
```
<br>

- Ir até o diretório do arquivo `app_local.py` e digitar no terminal:

```code
spark-submit app_local.py
```
